In [49]:
import pandas as pd

summary = []
summary.append({
    "GWAS": "GCST90450224", 
    "leads": "example_data/example=_leads.tsv", 
    "GWAS_path": "example_data/GCST90450224.tsv",
    "chr_col": "chromosome",
    "pos_col": "base_pair_location",
    "maf_col": "effect_allele_frequency",
    "beta_col": "beta",
    "se_col": "standard_error",
    "file_type": "tsv",
    "ref_col": "other_allele",
    "alt_col": "effect_allele"
    })

pd.DataFrame(summary).to_csv("example_gwas_table.tsv",sep="\t", index=False)

In [50]:
import pandas as pd

summary = []
summary.append({
    "id": "QTD000186", 
    "path": "example_data/QTD000186.lbf_variable.txt.gz.tsv",
    })

pd.DataFrame(summary).to_csv("example_eQTL_table.tsv",sep="\t", index=False)

In [48]:
import pandas as pd

df = pd.read_csv("GCST90450224.tsv", sep="\t")

df

,chromosome,base_pair_location,effect_allele,other_allele,beta,standard_error,effect_allele_frequency,neg_log_10_p_value,rsid,variant_id,info,ref_allele,n
0,1,285339,A,C,-0.020926,0.199508,0.014613,0.037884,rs879732627,1_285339_C_A,0.662782,OA,928
1,1,631712,C,T,0.161614,0.154556,0.017865,0.529126,rs9326622,1_631712_T_C,0.851058,OA,928
2,1,631739,C,A,0.161388,0.154579,0.017789,0.528031,rs9283153,1_631739_A_C,0.854172,OA,928
3,1,656627,A,G,-0.147961,0.125803,0.039830,0.620621,rs771664146,1_656627_G_A,0.661197,OA,928
4,1,656671,T,C,0.022934,0.175279,0.021045,0.047740,rs10218594,1_656671_C_T,0.609434,OA,928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
787942,1,248934255,A,C,0.131715,0.175541,0.013527,0.343852,rs138689967,1_248934255_C_A,0.953348,OA,928
787943,1,248934257,C,A,-0.114116,0.099312,0.046349,0.601138,rs199655021,1_248934257_A_C,0.903370,OA,928
787944,1,248936569,T,C,-0.047267,0.104786,0.038687,0.185798,rs532053890,1_248936569_C_T,0.877370,OA,928
787945,1,248936926,G,C,-0.088907,0.144758,0.019251,0.268333,rs199758136,1_248936926_C_G,0.948789,OA,928


In [37]:
# df = df[df["effect_allele_frequency"]>=0.05]

In [42]:
import numpy as np

def approx_bf_estimates(variant, z, V, type, suffix=None, sdY=1, effect_priors={'quant': 0.15, 'cc': 0.2}):
    sd_prior = effect_priors['quant'] * sdY if type == "quant" else effect_priors['cc']
    
    r = sd_prior**2 / (sd_prior**2 + V)
    lABF = 0.5 * (np.log(1 - r) + (r * z**2))
    
    ret = pd.DataFrame({"variant": variant, 'lbf': lABF})
    
    if suffix is not None:
        ret.columns = [f"{col}.{suffix}" for col in ret.columns]

    ret.loc[:, 'lbf'] = pd.to_numeric(ret['lbf'], errors='coerce')

    ret_mat = pd.DataFrame(ret.set_index('variant')['lbf']).T

    return ret_mat

df["variant"] = (
    "chr" + "1" + "_" +
    df["base_pair_location"].astype(int).astype(str) + "_" +
    df["other_allele"].astype(str) + "_" +
    df["effect_allele"].astype(str)
)

df["z"] = df["beta"] / df["standard_error"]
df["V"] = df["standard_error"] ** 2

mat = approx_bf_estimates(df["variant"],df["z"], df["V"], "quant") 
mat1 = mat.T
leads = mat1[mat1["lbf"]>=6].reset_index()
leads["position"] = leads["variant"].astype(str).str.split("_").str[1].astype(int)
leads

,variant,lbf,position
0,chr1_14826873_G_A,6.792095,14826873
1,chr1_76433270_G_A,6.541357,76433270
2,chr1_85516585_T_C,6.163146,85516585
3,chr1_85517720_A_G,6.057619,85517720
4,chr1_85543257_T_C,6.284835,85543257
5,chr1_218585370_G_A,6.246734,218585370
6,chr1_218588339_C_T,7.502817,218588339


In [43]:
leads["position"].unique()

array([ 14826873,  76433270,  85516585,  85517720,  85543257, 218585370,
       218588339])

In [45]:
leads_df= df[df["base_pair_location"].isin(leads["position"].unique())]
leads_df

leads_df.to_csv("example_leads.tsv", sep="\t")

In [ ]:
# 0.8973008	QTD000141_ENSG00000147403_L1	chrX_154356517_C_A	L_HDL_C_pct_chrX:153353564-155353564	chrX_154353564_A_G

In [67]:
import pandas as pd
df = pd.read_csv("QTD000141.lbf_variable.txt.gz", sep="\t")
df = df[(df["chromosome"]=="X")&(df["position"]>=153353564)&(df["position"]<=155353564)]
df.to_csv("eQTL_test.tsv", sep="\t", index=False)

/var/folders/g4/mx686c290ms3_t_t0prfdknc0000gn/T/ipykernel_87008/3956126524.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("QTD000141.lbf_variable.txt.gz", sep="\t")


In [69]:
import pandas as pd
df = pd.read_parquet("L_HDL_C_pct_EstBB_UKBB_EUR_metaanalysis.parquet")
# df = df[(df["chromosome"]==20)&(df["position"]>=62614455)&(df["position"]<=64332756)]
# max_row = df.loc[df["lbf_variable1"].idxmax()]

# print(max_row.variant)
df

,CHROM,GENPOS,ALLELE0,ALLELE1,Effect,StdErr,Direction,LOG10P,N,AC,AC_EstBB,AC_UKBB_EUR,INFO_EstBB,INFO_UKBB_EUR
0,1.0,14493688.0,C,A,0.046135,0.051013,++,0.436764,599249.0,342.603597,91.631356,250.972241,0.672614,0.926848
1,1.0,14493952.0,C,A,0.010776,0.068093,++,0.058359,599249.0,194.744826,80.831266,113.913560,0.873547,0.837506
2,1.0,14494530.0,A,G,0.002562,0.001738,++,0.852645,599249.0,369801.135506,129733.425360,240067.710146,0.998518,0.998861
3,1.0,14494565.0,A,G,-0.102877,0.075454,-?,0.762600,185352.0,151.141211,151.141211,0.000000,0.895840,NaN
4,1.0,14494916.0,G,A,-0.005392,0.004950,--,0.559116,599249.0,32317.712350,10317.100094,22000.612256,0.991901,0.997938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103927049,23.0,155700123.0,C,A,-0.042162,0.100167,?-,0.171458,412523.0,72.121561,0.000000,72.121561,NaN,0.706265
103927050,23.0,155700124.0,A,G,-0.042162,0.100167,?-,0.171458,412523.0,72.121561,0.000000,72.121561,NaN,0.706265
103927051,23.0,155700291.0,CAA,C,0.010731,0.001967,?+,7.312481,412523.0,190065.021974,0.000000,190065.021974,NaN,0.915101
103927052,23.0,155700569.0,A,G,0.010767,0.001967,?+,7.356952,412523.0,190100.498952,0.000000,190100.498952,NaN,0.915012


In [73]:
df = df[(df["CHROM"].astype(int)==23)&(df["GENPOS"]>=153353564)&(df["GENPOS"]<=155353564)]
df.to_csv("gwas_test.tsv", sep="\t", index=False)

In [79]:
df = pd.read_csv("meta_EUR_all_lead_variants.tsv", sep="\t")
df = df[df["metabolite"]=="L_HDL_C_pct"]
df = df[(df["CHR"].astype(int)==23)&(df["POS"]>=153353564)&(df["POS"]<=155353564)]
df.to_csv("test_lead.tsv", sep="\t")

In [80]:
df = pd.read_parquet("/Users/mihkeljesse/Desktop/metabolite-coloc/example/formatted_eqtls/X/chrX_met_group_0_region_153354900-155341332.parquet")
df

,signal,chromosome,location_min,location_max,signal_strength,lead_variant,chrX_153354900_T_C,chrX_153355215_C_T,chrX_153356562_A_T,chrX_153357170_C_T,...,chrX_155329596_C_T,chrX_155332643_TTAA_T,chrX_155333588_C_T,chrX_155335094_A_T,chrX_155335631_C_G,chrX_155336095_T_C,chrX_155336687_G_A,chrX_155340984_G_C,chrX_155341314_C_A,chrX_155341332_C_T
0,QTD000141_ENSG00000183479_L2,X,153354900,154469313,9.218460,chrX_153501567_C_T,-0.881082,-0.637445,-1.411161,-1.456741,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
1,QTD000141_ENSG00000183479_L1,X,153354900,154469313,20.285905,chrX_153463824_A_G,-1.279783,0.041625,-1.791401,-1.846518,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
2,QTD000141_ENSG00000063587_L1,X,153354900,154334108,38.016851,chrX_153358607_A_G,-1.293147,-0.597093,6.498758,5.856853,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
3,QTD000141_ENSG00000147394_L1,X,153354900,153913475,5.053577,chrX_153697218_G_T,-1.781913,-1.229357,-1.657736,-2.008042,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
4,QTD000141_ENSG00000177854_L1,X,153354900,154971981,129.435095,chrX_153946200_A_G,-1.049764,-2.744284,-2.751944,-2.680944,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
5,QTD000141_ENSG00000235961_L1,X,153354900,154070060,6.021715,chrX_153771272_G_C,-2.127561,-2.273388,-1.892488,-2.041216,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
6,QTD000141_ENSG00000232725_L2,X,153354900,154764114,16.180120,chrX_153982797_G_A,-0.896369,-0.977183,-0.941568,-0.890753,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
7,QTD000141_ENSG00000067840_L1,X,153354900,154829179,25.077366,chrX_153828330_G_C,-1.963034,-1.714218,-1.715277,-1.598115,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
8,QTD000141_ENSG00000189420_L1,X,153354900,154410919,28.336466,chrX_153434750_T_C,-1.714447,-1.927175,-1.923859,-1.932311,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
9,QTD000141_ENSG00000198753_L1,X,153354900,154764114,74.226748,chrX_153952914_C_A,-2.205078,-2.513901,-2.131955,-1.671441,...,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000,-1000000.000000
